## Movies Rating Scraping from IMDb

In [2]:
%pip install requests beautifulsoup4 pandas time

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\hchai\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import requests

from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
# List of URLs of the website to scrape
url = "https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page="
urls = []

for i in range(1,9):
    urls.append(url+str(i))

urls

['https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=1',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=2',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=3',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=4',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=5',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=6',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=7',
 'https://www.imdb.com/list/ls000634294/?sort=list_order,asc&st_dt=&mode=detail&page=8']

In [5]:
# The scraping function
def scrape(url):
    # Send an HTTP GET request to the website
    response = requests.get(url)

    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the relevant information from the HTML code
    # Loop through each item in the movies list
    for item in soup.select('div.lister-list div.lister-item'):
        title = item.find('div', class_='lister-item-content').find('h3', class_='lister-item-header').find('a').get_text()
        year = item.find('div', class_='lister-item-content').find('h3', class_='lister-item-header').find('span', class_='lister-item-year').get_text()
        runtime = item.find('div', class_='lister-item-content').find('p', class_='text-muted').find('span', class_='runtime').get_text()
        genre = item.find('div', class_='lister-item-content').find('p', class_='text-muted').find('span', class_='genre').get_text()
        rating = item.find('div', class_='lister-item-content').find('div', class_='ipl-rating-widget').find('div', class_='ipl-rating-star').find('span', class_='ipl-rating-star__rating').get_text()
        movies.append([title, year, runtime, genre,rating])

In [6]:
# Initializing the movies list
movies = []

# Scraping the pages
for url in urls:
    scrape(url)

# Storing the information in a pandas dataframe
df = pd.DataFrame(movies, columns=['Title', 'Year', 'Runtime', 'Genre', 'Rating'])

# Adding a delay between requests to avoid overwhelming the website with requests
time.sleep(1)

In [7]:
df.shape

(787, 5)

In [8]:
df.head()

,Title,Year,Runtime,Genre,Rating
0,The Shawshank Redemption,(1994),142 min,\nDrama,9.3
1,The Godfather,(1972),175 min,"\nCrime, Drama",9.2
2,Pulp Fiction,(1994),154 min,"\nCrime, Drama",8.9
3,The Godfather Part II,(1974),202 min,"\nCrime, Drama",9
4,Inception,(2010),148 min,"\nAction, Adventure, Sci-Fi",8.8


In [12]:
# Eliminating the () from year 
df['Year'] = df['Year'].str.replace(r'[()]', '', regex=True)

# Deleting min from Runtime
df['Runtime'] = df['Runtime'].str.replace(" min", '', regex=True)

# Deleting the \n from Genre
df['Genre'] = df['Genre'].str.replace(r'[\n]', '', regex=True)

# Removing spaces from Genre
df['Genre'] = df['Genre'].str.replace(r'[            ]', '', regex=True)


In [13]:
df.head()

,Title,Year,Runtime,Genre,Rating
0,The Shawshank Redemption,1994,142,Drama,9.3
1,The Godfather,1972,175,"Crime,Drama",9.2
2,Pulp Fiction,1994,154,"Crime,Drama",8.9
3,The Godfather Part II,1974,202,"Crime,Drama",9
4,Inception,2010,148,"Action,Adventure,Sci-Fi",8.8


In [14]:
df.to_csv('./data/IMDb-movies-rating.csv')